<a href="https://colab.research.google.com/github/TinaKhatri28/ASTRA-Project/blob/main/model3final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/sih project/sih dataset"

healthy  infected


In [ ]:
import os
import cv2
import numpy as np
import shutil
from tqdm import tqdm

SOURCE_DIR = "/content/drive/MyDrive/sih project/sih dataset/infected"
OUTPUT_DIR = "/content/drive/MyDrive/sih_project/model3_severity_dataset"

# Make output folders
for cls in ["mild", "moderate", "severe"]:
    os.makedirs(os.path.join(OUTPUT_DIR, cls), exist_ok=True)

def detect_infection_percentage(img):
    # Resize small
    img = cv2.resize(img, (300, 300))

    # Convert to HSV to detect brown/yellow disease areas
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Brown/yellow/pale disease color range
    lower = np.array([5, 30, 20])
    upper = np.array([35, 255, 255])

    mask = cv2.inRange(hsv, lower, upper)

    infected_pixels = np.sum(mask > 0)
    total_pixels = mask.size

    return (infected_pixels / total_pixels) * 100


images = [f for f in os.listdir(SOURCE_DIR)
          if f.lower().endswith((".jpg", ".jpeg", ".png"))]

print(" Total Images Found:", len(images))

for imgname in tqdm(images):
    path = os.path.join(SOURCE_DIR, imgname)
    img = cv2.imread(path)
    if img is None:
        continue

    percent = detect_infection_percentage(img)

    # Classification based on % infected area
    if percent < 20:
        dest = "mild"
    elif percent < 60:
        dest = "moderate"
    else:
        dest = "severe"

    shutil.move(path, os.path.join(OUTPUT_DIR, dest, imgname))


 Total Images Found: 0


0it [00:00, ?it/s]


In [ ]:
DATASET_PATH ="/content/drive/MyDrive/sih_project/model3_severity_dataset"


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.20
)
# data training
train_data = train_gen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    subset="training"
)

val_data = train_gen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    subset="validation"
)

print("\n Class Mapping:", train_data.class_indices)


Found 5419 images belonging to 3 classes.
Found 1353 images belonging to 3 classes.

 Class Mapping: {'mild': 0, 'moderate': 1, 'severe': 2}


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(3, activation='softmax')  # mild, moderate, severe
])

model.compile(
    optimizer=Adam(0.0005),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
callbacks = [
    ModelCheckpoint("model3_best_severity.h5", save_best_only=True, monitor="val_accuracy", mode="max"),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3)
]


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=callbacks
)



print(" MODEL 3 TRAINING COMPLETED!")


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.6227 - loss: 0.8920

170/170 ━━━━━━━━━━━━━━━━━━━━ 1934s 11s/step - accuracy: 0.6229 - loss: 0.8914 - val_accuracy: 0.6970 - val_loss: 0.6946 - learning_rate: 5.0000e-04
Epoch 2/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.7057 - loss: 0.6871 - val_accuracy: 0.6962 - val_loss: 0.6846 - learning_rate: 5.0000e-04
Epoch 3/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7186 - loss: 0.6518

170/170 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.7185 - loss: 0.6519 - val_accuracy: 0.7051 - val_loss: 0.6705 - learning_rate: 5.0000e-04
Epoch 4/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7343 - loss: 0.6196

170/170 ━━━━━━━━━━━━━━━━━━━━ 270s 2s/step - accuracy: 0.7343 - loss: 0.6197 - val_accuracy: 0.7066 - val_loss: 0.6619 - learning_rate: 5.0000e-04
Epoch 5/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7419 - loss: 0.6316

...

170/170 ━━━━━━━━━━━━━━━━━━━━ 239s 1s/step - accuracy: 0.7670 - loss: 0.5629 - val_accuracy: 0.7243 - val_loss: 0.6368 - learning_rate: 5.0000e-04
Epoch 10/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7656 - loss: 0.5627

170/170 ━━━━━━━━━━━━━━━━━━━━ 291s 2s/step - accuracy: 0.7656 - loss: 0.5627 - val_accuracy: 0.7258 - val_loss: 0.6580 - learning_rate: 5.0000e-04
Epoch 11/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.7746 - loss: 0.5311 - val_accuracy: 0.7236 - val_loss: 0.6396 - learning_rate: 5.0000e-04
Epoch 12/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 270s 2s/step - accuracy: 0.7858 - loss: 0.5188 - val_accuracy: 0.7236 - val_loss: 0.6245 - learning_rate: 1.5000e-04
Epoch 13/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 258s 2s/step - accuracy: 0.7767 - loss: 0.5269 - val_accuracy: 0.7199 - val_loss: 0.6406 - learning_rate: 1.5000e-04
Epoch 14/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 265s 2s/step - accuracy: 0.7861 - loss: 0.5177 - val_accuracy: 0.7258 - val_loss: 0.6281 - learning_rate: 1.5000e-04
Epoch 15/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 264s 2s/step - accuracy: 0.7849 - loss: 0.5150 - val_accuracy: 0.7258 - val_loss: 0.6139 - learning_rate: 1.5000e-04
Epoch 16/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 

170/170 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.7882 - loss: 0.5086 - val_accuracy: 0.7332 - val_loss: 0.6332 - learning_rate: 1.5000e-04
Epoch 17/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 264s 2s/step - accuracy: 0.7983 - loss: 0.4866 - val_accuracy: 0.7317 - val_loss: 0.6156 - learning_rate: 1.5000e-04
Epoch 18/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7855 - loss: 0.5036

170/170 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - accuracy: 0.7855 - loss: 0.5036 - val_accuracy: 0.7347 - val_loss: 0.6282 - learning_rate: 1.5000e-04
Epoch 19/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 296s 2s/step - accuracy: 0.7974 - loss: 0.4964 - val_accuracy: 0.7339 - val_loss: 0.6246 - learning_rate: 4.5000e-05
Epoch 20/20
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7957 - loss: 0.4754

170/170 ━━━━━━━━━━━━━━━━━━━━ 266s 2s/step - accuracy: 0.7957 - loss: 0.4755 - val_accuracy: 0.7435 - val_loss: 0.6226 - learning_rate: 4.5000e-05
 MODEL 3 TRAINING COMPLETED!


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model = load_model("/content/model3_best_severity.h5")

CLASS_MAP = {0: "mild", 1: "moderate", 2: "severe"}

def predict_severity(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)[0]
    severity = CLASS_MAP[np.argmax(pred)]
    confidence = np.max(pred)

    print(f"Severity Level: {severity} (Confidence: {confidence:.2f})")
img_path = "/content/drive/MyDrive/sih_project/model3_severity_dataset/mild/0cashew_train_red rust.JPG"
if os.path.exists(img_path):
    predict_severity(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Severity Level: mild (Confidence: 0.84)


In [ ]:
model.save("/content/drive/MyDrive/sih_project/model3_final_severity.h5")